In [ ]:
import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import os
from math import log, floor
from scipy import signal
from scipy.signal import savgol_filter
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from scipy.io import savemat
from collections import Counter
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import scipy.stats as stats
import math
import pandas as pd
from scipy.signal import resample
import seaborn as sns
from scipy.fft import fft, fftfreq
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score
import numpy as np
from scipy import signal
from scipy.ndimage import shift
import matplotlib.pyplot as plt
import sys

import pywt

from sklearn.mixture import GaussianMixture
from multiprocessing import Pool
import time
from multiprocessing import Pool
from tqdm.autonotebook import tqdm
import ex_features
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import moving_avg
from starter_func import *
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
%load_ext autoreload
%autoreload 2

## Loading Data 

In [ ]:
dir_path = " "
main_dir = f"{dir_path}/Data" 
vm_trace, ca_trace = data_load(main_dir=main_dir) 

## Event Detection 

### Average Subtraction 

In [ ]:
if __name__ == '__main__':
    start_time = time.perf_counter()
    with Pool(15) as p:
        avg_subtracted_par = list(tqdm(p.imap(moving_avg.moving_avg_subtraction, vm_trace), total=len(vm_trace)))
    finish_time = time.perf_counter()
    print("Program finished in {} seconds - using serial processing".format(finish_time-start_time))

### Peak Detection 

In [ ]:
peaks_comb = peak_sort_(smooth_vm=vm_trace)

### Onset & End Detection 

In [ ]:
onsets, ends = onset_end_func(avg_subtracted=avg_subtracted_par, peaks_comb=peaks_comb)
unique_on = [unique_preserve_order(a) for a in onsets]
unique_en = [unique_preserve_order(a) for a in ends]


In [ ]:
#removing any false positive APs
event_dict = [{'onsets': onsets, 'ends': ends}]
filtered_events = remove_false_positives(vm_trace, event_dict, threshold=-20)

# Extract onsets and ends from the first dictionary (assuming 1 signal)


In [ ]:
onset_ = [event['onsets'] for event in filtered_events]
end_ = [event['ends'] for event in filtered_events]

### Recording Plot Example (Original)

In [ ]:
plt.figure(figsize=(20, 6))
plt.rcParams.update({'font.size': 14})
n = 90
t = np.linspace(0, 100, len(vm_trace[n]))
fs = len(vm_trace[n])/100
plt.plot(t, vm_trace[n], color = 'black')
plt.xlabel("Time (s)")
plt.ylabel("Potential (mV)")
plt.ylim(-80, 30)
plt.show()

### Recording Plot Example (Onset & End marked) 

In [ ]:
plt.figure(figsize=(20, 6))
plt.rcParams.update({'font.size': 14})
n = 90
t = np.linspace(0, 90, len(avg_subtracted_par[n]))
#t = np.linspace(0, 100, len(vm_trace[70]))
fs = len(avg_subtracted_par[n])/90
#fs = len(avg_subtracted_par[20])/100
plt.plot(t, avg_subtracted_par[n], color = 'black')
#plt.plot(t, vm_trace[70], color = 'black')
plt.xlabel("Time (s)")
plt.ylabel("Potential (mV)")
#plt.xlim(0, 95)
plt.ylim(-20, 100)
for start, end in zip(onset_[n][:-1], end_[n][:-1]):
    plt.axvline(x=start/fs, color='green', linestyle='-', linewidth=1)
    plt.axvline(x= end/fs, color='red', linestyle='-', linewidth=1)

### Extract Events 

In [ ]:
events = extract_events_unmerged(vm_trace, onset_, end_, 'vm') #membrane voltage events 
ca_events = extract_events_unmerged(ca_trace, onset_, end_, 'ca') #calcium imaging events 

In [ ]:
#Calcium signal with added noise for further analysis
sig = 0.5*0.007
ca_events_gauss = add_noise_to_signals(ca_events, noise_type='gaussian', sigma=sig)
ca_events_sp = add_noise_to_signals(ca_events, noise_type='salt_and_pepper')


## Feature Extraction 

In [ ]:
## Frequency Features 

In [ ]:
features_vm_freq = extract_features_freq(unique_events) #membrane potential frequency features 
len(features_vm_freq)

In [ ]:
features_ca_freq = extract_features_freq(ca_unique_events) #calcium activity frequency features 
len(features_ca_freq)

In [ ]:
features_vm_freq = np.array(features_vm_freq)
features_ca_freq = np.array(features_ca_freq)

In [ ]:
####To plot the length of the detected events#####

event_len = []

for i in range(0, len(unique_events[0])):
    for j in range(0, len(unique_events[i])):
        event_len.append(len(unique_events[i][j]))
    
event_len = np.array(event_len)
event_len = event_len/10 #event length calculated in ms. Approx fs taken as 1000 
event_len_log = np.log(event_len)
len_mean = np.mean(event_len)
len_median = np.median(event_len)
num_bins = 50

plt.hist(event_len, bins=num_bins, color = 'green') 
plt.axline((len_mean, 0), slope = np.inf, color= 'red', label = 'Mean Event Length')
plt.axline((len_median, 0), slope = np.inf, color= 'black', label = 'Median Event Length')
plt.legend(loc='best', fontsize='small', title='', frameon=True, shadow=True, fancybox=True)
plt.xlabel("Event Length (ms)")
plt.ylabel("Number of Events")
plt.ylim(0, 100)
plt.show()

## Clustering 

In [ ]:
gmm_clus_analysis(np.array(features_vm_freq))

In [ ]:
print(features_vm_freq.shape)
print(features_vm_freq_edited.shape)

In [ ]:
gmm = GaussianMixture(n_components=7, random_state=0)
gmm.fit(features_vm_freq)
original_clusters = gmm.fit_predict(features_vm_freq)

In [ ]:
probabilities = gmm.predict_proba(features_vm_freq)
labels = assign_labels_with_threshold(probabilities)

In [ ]:
frequency = Counter(labels)
print(frequency)

## Classifier 

In [ ]:
##Accuracy for GMM
X_train, X_test, y_train, y_test = train_test_split(features_ca_freq, labels, test_size=0.4, random_state=42)


rf_clf = RandomForestClassifier(n_estimators=1000, random_state=42)


rf_clf.fit(X_train, y_train)


y_pred = rf_clf.predict(X_test)


from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
from sklearn.model_selection import cross_val_score
rf_claf = RandomForestClassifier(n_estimators=100, random_state=42)
cv_scores = cross_val_score(rf_claf, X_train, y_train, cv=5)

# Output the results
print("Cross-validation scores:", cv_scores)
print("Average score:", np.mean(cv_scores))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# For classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (confusion_matrix, classification_report,
                             accuracy_score, matthews_corrcoef, cohen_kappa_score)

cm = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(cm)

# 2. Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 3. Additional Metrics

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.3f}".format(accuracy))

# Matthews Correlation Coefficient (MCC)
mcc = matthews_corrcoef(y_test, y_pred)
print("Matthews Correlation Coefficient (MCC): {:.3f}".format(mcc))

# Cohen's Kappa
kappa = cohen_kappa_score(y_test, y_pred)
print("Cohen's Kappa: {:.3f}".format(kappa))

# 4. Plotting the Confusion Matrix
plt.figure(figsize=(6, 5))
plt.imshow(cm, interpolation='nearest', aspect='auto')
plt.title("Confusion Matrix")
plt.colorbar()
num_classes = cm.shape[0]
tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, tick_marks)
plt.yticks(tick_marks, tick_marks)
plt.ylabel('True label')
plt.xlabel('Predicted label')

# Add text annotations inside the cells
thresh = cm.max() / 2.0
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(j, i, format(cm[i, j], 'd'),
                 ha="center", va="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.show()

In [ ]:
import shap

explainer = shap.Explainer(rf_clf, X_train)
shap_values = explainer(X_test)

plt.figure(figsize=(15,6))
shap.summary_plot(shap_values, X_test)


## Feature Plots 

In [ ]:
from scipy.stats import shapiro
num_columns = 9

list_ = ['Max (Power Spectrum, FT) ','Mean (Power Spectrum, FT) ','Skewness (Power Spectrum, FT) ','Kurtosis (Power Spectrum, FT)',  
              'Mean (DWT coeffs)', 'Power (DWT coeffs)', 'STD (DWT coeffs)', 'Skewness (DWT coeffs)'
            , 'Kurtosis (DWT coeffs)']
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 15))
for i in range(num_columns):
    ax = axes[i // 3, i % 3]
    ax.hist(features_vm_freq_edited[:, i], bins=500, alpha=0.7, color='blue')
    #ax.set_xlabel(f'Column {i+1} Values')
    ax.set_ylabel('Frequency')
    stat, p = shapiro(features_vm_freq_edited[:, i])
    ax.text(0.5, 0.8, f"{stat:.3f}", color='black', fontsize=12, transform=ax.transAxes)
    ax.text(0.5, 0.7, f"p = {p:.2e}", color='black', fontsize=12, transform=ax.transAxes)
    ax.set_title(list_[i])
    #plt.show()
    # 
    # print(stat, p)
plt.tight_layout()
plt.show()

## More Plots

In [ ]:
vm_labels, num_events = create_labels(unique_events) #outputs the labels [file number] for each event and number of events detected in each file 

In [ ]:
neuron_labels = []

# Dictionary to map unique experiment numbers to labels
experiment_map = {}

# Counter for labeling experiments
label_counter = 0

items = os.listdir(main_dir)
file_names = sorted(items)
# Loop through each file name
for file in file_names:
    if file.endswith('.npz'):
        # Extract the first 13 characters from the file name
        experiment = file[:13]
        #print(experiment)
        # Check if the experiment is already in the map
        if experiment not in experiment_map:
            # If not, assign it the next available label
            experiment_map[experiment] = label_counter
            label_counter += 1
        
        # Append the label to the experiment_labels list
        neuron_labels.append(experiment_map[experiment])

# Display the experiment labels list
#print(neuron_labels) # represents experiments according to neuron labels {45 neurons in total} 
#len(neuron_labels)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

def compute_power_spectrum(signal, fs=1.0):
    """
    Computes the power spectrum of a given 1D signal using FFT.
    
    Parameters:
    -----------
    signal : array-like
        Time-domain signal.
    fs : float
        Sampling frequency (samples per unit time).
        
    Returns:
    --------
    freq : np.ndarray
        Array of frequency bins (from 0 to fs/2 for rfft).
    power : np.ndarray
        Power at each frequency bin.
    """
    n = len(signal)
    # Frequencies for the real-valued FFT (rfft)
    freq = np.fft.rfftfreq(n, d=1.0/fs)
    # Compute the FFT of the signal
    fft_vals = np.fft.rfft(signal)
    # Power is magnitude squared, normalized by the signal length
    power = (np.abs(fft_vals) ** 2) / n
    return freq, power

def plot_group_power_spectra(signals, labels, fs=8550.0, num_groups=7):
    """
    Plots the average power spectrum for each group label.
    
    Parameters:
    -----------
    signals : list of np.ndarray
        List of 1D signals (each can be of varying length).
    labels : list or np.ndarray of int
        List of group labels (e.g., integers 0 through 6), 
        same length as signals.
    fs : float
        Sampling frequency for the signals.
    num_groups : int
        Total number of possible groups (e.g., 7 if labels range 0..6).
    """
    # Group signals by their labels
    group_signals = defaultdict(list)
    for sig, lbl in zip(signals, labels):
        group_signals[lbl].append(sig)
    
    plt.figure(figsize=(10, 6))
    
    # For each group, compute and plot the average power spectrum
    coun = 0
    for g in range(7):
        if len(group_signals[g]) == 0:
            
            continue  # Skip groups with no signals
        
        power_spectra = []
        for signal in group_signals[g]:
            if len(signal) < 200:
                coun = coun + 1
                continue 
            freq, power = welch(signal, fs=fs, nperseg=200, noverlap=50)
            power_spectra.append(power)
        
        # Compute the average power across all signals in the group
        avg_power = np.mean(power_spectra, axis=0)
        
        # Plot
        plt.plot(freq, avg_power, label=f"Group {g}")
    
    plt.title("Power Spectra by Group")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power")
    plt.legend()
    plt.xlim(0,500)
    plt.show()
    print(coun)
# Example usage:
if __name__ == "__main__":
    # Example signals (varying lengths)
    # Example labels (0-6), same length as 'signals'
    events_concat = []
    for ex in unique_events:
        for sig in ex:
            events_concat.append(np.array(sig))
            
    # Plot the power spectra
    plot_group_power_spectra(events_concat, labels, fs=8550.0, num_groups=7)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

def compute_power_spectrum(signal, fs=1.0):
    """
    Computes the power spectrum of a given 1D signal using FFT.
    
    Parameters:
    -----------
    signal : array-like
        Time-domain signal.
    fs : float
        Sampling frequency (samples per unit time).
        
    Returns:
    --------
    freq : np.ndarray
        Array of frequency bins (from 0 to fs/2 for rfft).
    power : np.ndarray
        Power at each frequency bin.
    """
    n = len(signal)
    # Frequencies for the real-valued FFT (rfft)
    freq = np.fft.rfftfreq(n, d=1.0/fs)
    # Compute the FFT of the signal
    fft_vals = np.fft.rfft(signal)
    # Power is magnitude squared, normalized by the signal length
    power = (np.abs(fft_vals) ** 2) / n
    return freq, power

def plot_group_power_spectra(signals, labels, fs=8550.0, num_groups=7):
    """
    Plots the average power spectrum for each group label.
    
    Parameters:
    -----------
    signals : list of np.ndarray
        List of 1D signals (each can be of varying length).
    labels : list or np.ndarray of int
        List of group labels (e.g., integers 0 through 6), 
        same length as signals.
    fs : float
        Sampling frequency for the signals.
    num_groups : int
        Total number of possible groups (e.g., 7 if labels range 0..6).
    """
    # Group signals by their labels
    group_signals = defaultdict(list)
    for sig, lbl in zip(signals, labels):
        group_signals[lbl].append(sig)
    
    plt.figure(figsize=(10, 6))
    
    # For each group, compute and plot the average power spectrum
    coun = 0
    g = 0
    # if len(group_signals[g]) == 0:
        
    #       # Skip groups with no signals
    
    power_spectra = []
    for signal in group_signals[g]:
        if len(signal) < 200:
            coun = coun + 1
            continue 
        freq, power = welch(signal, fs=fs, nperseg=200, noverlap=50)
        power_spectra.append(power)
        plt.plot(power)
    
    # Compute the average power across all signals in the group
    avg_power = np.mean(power_spectra, axis=0)
    
    # Plot
    plt.plot(avg_power, label=f"Group {g}")
    
    plt.title("Power Spectra by Group")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power")
    plt.legend()
    plt.xlim(0,10)
    plt.show()
    print(coun)
# Example usage:
if __name__ == "__main__":
    # Example signals (varying lengths)
    # Example labels (0-6), same length as 'signals'
    events_concat = []
    for ex in unique_events:
        for sig in ex:
            events_concat.append(np.array(sig))
            
    # Plot the power spectra
    plot_group_power_spectra(events_concat, labels, fs=8550.0, num_groups=7)